In [40]:
import os
from amplpy import AMPL, Environment

**File Paths**

In [41]:
baseexppath = '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/'
amplappdir = os.path.join(baseexppath, 'ampl/amplide.macosx64/')

# Specify model and data files
# f_mod = os.path.join(baseexppath, 'ampl/example/steel3.mod')
f_mod = os.path.join(baseexppath, 'ampl/test4.mod')
# f_dat = os.path.join(baseexppath, 'ampl/example/steel3.dat')
f_dat = os.path.join(baseexppath, 'ampl/test4.dat')

# Specify AMPL solver to be used
f_minos_solver = os.path.join(baseexppath, 'ampl/amplide.macosx64/minos')
f_gurobi_solver = os.path.join(baseexppath, 'ampl/amplide.macosx64/gurobi')
f_solver = f_minos_solver

# Data table directories
sourcedatadir = os.path.join(baseexppath, 'OptSandbox/data/test_source/')
metadatadir = os.path.join(baseexppath, 'OptSandbox/data/test_metadata/')

**Set up the AMPL environment**

In [42]:
ampl = AMPL(Environment(amplappdir))
ampl.setOption('solver', f_solver)
value = ampl.getOption('solver')
print(value)

/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/ampl/amplide.macosx64/minos


**Read in the model**

In [43]:
ampl.read(f_mod)

**Read in the data**

In [50]:
import pandas as pd
from IPython.display import display

In [65]:
# Read in the data tables
TblBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblBmp.csv'))
TblBmpGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpGroup.csv'))
TblBmpLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpLoadSourceGroup.csv'))
TblBmpType = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpType.csv'))
TblLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSource.csv'))
TblLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroup.csv'))
TblLoadSourceGroupLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroupLoadSource.csv'))

**SETS**

In [ ]:
# sets:
# PLNTS
# LRSEGS
pltnts = ampl.getSet('PLTNTS')
pltnts.setValues(['N', 'P', 'S'])
print(pltnts.getValues())

lrsegs = ampl.getSet('LRSEGS')
lrsegs.setValues(['N51133RL0_6450_0000'])
print(lrsegs.getValues())

In [83]:
# sets:
# LOADSRCS
# LOADSRCGRPS
# LOADSRCGRPING
loadsrcs = ampl.getSet('LOADSRCS')
loadsrcs.setValues(TblLoadSource.loadsourceid.tolist())
#print(loadsrcs.getValues())

loadsrcgrps = ampl.getSet('LOADSRCGRPS')
loadsrcgrps.setValues(TblLoadSourceGroup.loadsourcegroupid.tolist())
#print(loadsrcgrps.getValues())

loadsrcgrping = ampl.getSet('LOADSRCGRPING')
loadsrcgrping.setValues(list(zip(TblLoadSourceGroupLoadSource.loadsourcegroupid.tolist(),
                             TblLoadSourceGroupLoadSource.loadsourceid.tolist())))
#print(loadsrcgrping.getValues())

In [82]:
# sets:
# BMPS
# BMPGRPS
# BMPGRPING
bmps = ampl.getSet('BMPS')
bmps.setValues(TblBmp.bmpid.tolist())
# print(bmps.getValues())

bmpgrps = ampl.getSet('BMPGRPS')
bmpgrps.setValues(TblBmpGroup.bmpgroupid.tolist())
# print(bmpgrps.getValues())

bmpgrping = ampl.getSet('BMPGRPING')
bmpgrping.setValues(list(zip(TblBmp.bmpid.tolist(),
                             TblBmp.bmpgroupid.tolist())))
# print(bmpgrping.getValues())

**PARAMETERS**

In [ ]:
# Read in the data tables
TblCostBmpLand = pd.read_csv(os.path.join(metadatadir, 'TblCostBmpLand.csv'))
TblBmpEfficiency = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpEfficiency.csv'))
# Target load reductions ???  (set this ourselves??)
TblLandUsePreBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblLandUsePreBmp.csv'))


In [84]:
ampl.readData(f_dat)

Error: data for PLTNTS already read
Error: data for LRSEGS already read
Error: data for LOADSRCS already read
Error: data for BMPS already read


RuntimeError: Errors: 4; Warnings: 0

**Solve the problem**

In [14]:
ampl.solve()

Gurobi 8.0.0: optimal solution; objective 194828.5714
1 simplex iterations
